In [1]:
%run ../__init__.py

## Data preprocessing with Tfidf

In [2]:
budget_df = pd.read_csv('../data/budget_df_cleaned1.csv', index_col=0)

In [3]:
# budget_df.head()

In [4]:
budget_test = pd.read_csv('../data/budget_test_clean.csv', index_col=0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
for col in budget_test[word_predictors[0:14]]:
    budget_test[col] = budget_test[word_predictors[0:14]].loc[:,[col]].fillna(' ')

In [6]:
for col in budget_df[word_predictors]:
    budget_df[col] = budget_df[word_predictors].loc[:,[col]].fillna(' ')

In [7]:
test_cols = ['Object_Description','Text_2','SubFund_Description','Job_Title_Description','Text_3','Text_4',
                   'Sub_Object_Description','Location_Description','Function_Description','Facility_or_Department','Position_Extra',
                   'Program_Description','Fund_Description','Text_1']

In [48]:
def create_tfidf(word_column):
    tfidf = TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)
    col_tfidf = tfidf.fit_transform(budget_df[word_column])
    
    return col_tfidf

In [49]:
Object_Description_tfidf = create_tfidf('Object_Description')
Text_2_tfidf= create_tfidf('Text_2')
SubFund_Description_tfidf = create_tfidf('SubFund_Description')
Job_Title_Description_tfidf = create_tfidf('Job_Title_Description')
Text_3_tfidf = create_tfidf('Text_3')
Text_4_tfidf= create_tfidf('Text_4')
Sub_Object_Description_tfidf= create_tfidf('Sub_Object_Description')
Location_Description_tfidf= create_tfidf('Location_Description')
Function_Description_tfidf= create_tfidf('Function_Description')
Facility_or_Department_tfidf= create_tfidf('Facility_or_Department')
Position_Extra_tfidf= create_tfidf('Position_Extra')
Program_Description_tfidf= create_tfidf('Program_Description')
Fund_Description_tfidf= create_tfidf('Fund_Description')
Text_1_tfidf= create_tfidf('Text_1')

In [10]:
word_predictors.append('combined_tfidif')

In [11]:
from scipy import sparse
from scipy.sparse import hstack

combined_tfidf = hstack([Object_Description_tfidf, Text_2_tfidf, SubFund_Description_tfidf, Job_Title_Description_tfidf,
                         Text_3_tfidf, Text_4_tfidf,Sub_Object_Description_tfidf, Location_Description_tfidf,
                         Function_Description_tfidf,Facility_or_Department_tfidf, Position_Extra_tfidf,
                         Program_Description_tfidf, Fund_Description_tfidf, Text_1_tfidf])

In [12]:
tfidf_list = [Object_Description_tfidf, Text_2_tfidf, SubFund_Description_tfidf, Job_Title_Description_tfidf, Text_3_tfidf,\
              Text_4_tfidf,Sub_Object_Description_tfidf, Location_Description_tfidf, Function_Description_tfidf,\
              Facility_or_Department_tfidf, Position_Extra_tfidf, Program_Description_tfidf,\
              Fund_Description_tfidf, Text_1_tfidf, combined_tfidf]

In [13]:
len(word_predictors), len(tfidf_list)

(15, 15)

## Model Predictions for Operating_status

In [14]:
from sklearn.metrics import log_loss

from sklearn.metrics.scorer import make_scorer

log_loss_scorer = make_scorer(log_loss)

In [15]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [16]:
def nlp_pipe(target_name, predictor_name, X, y):
     
    X_train, X_val, y_train, y_val = train_test_split(X, 
                                                    y, 
                                                    test_size=.2, 
                                                    random_state=42)
        
    OVR = OneVsRestClassifier(LogisticRegression(random_state=42))

    
    model = OVR.fit(X_train, y_train)
#     add log_loss here 
    
    results = {'target': target_name,
               'predictor': predictor_name,
               'train_score': model.score(X_train, y_train),
               'val_score': model.score(X_val, y_val)}
    
    return model, results

In [17]:
op_status_results = []
for tfidf, predictor in zip(tfidf_list, word_predictors):  
    results = nlp_pipe('Operating Status', predictor, tfidf, le.fit_transform(budget_df['Operating_Status']))
    op_status_results.append(results)

In [43]:
budget_df['Operating_Status'].value_counts()/budget_df['Operating_Status'].shape

Operating_Status__PreK-12 Operating         0.858351
Operating_Status__Non-Operating             0.120002
Operating_Status__Operating, Not PreK-12    0.021648
Name: Operating_Status, dtype: float64

In [39]:
# op_status_results

In [18]:
score_list = []
for item in op_status_results:
    scores = item[1]
    score_list.append(scores)

In [19]:
pd.DataFrame(score_list)

,predictor,target,train_score,val_score
0,Object_Description,Operating Status,0.919721,0.918170
1,Text_2,Operating Status,0.875876,0.874438
2,SubFund_Description,Operating Status,0.934002,0.932909
3,Job_Title_Description,Operating Status,0.865412,0.864245
4,Text_3,Operating Status,0.858560,0.857512
5,Text_4,Operating Status,0.858798,0.857862
6,Sub_Object_Description,Operating Status,0.920036,0.919007
7,Location_Description,Operating Status,0.878212,0.876924
8,Function_Description,Operating Status,0.942034,0.940429
9,Facility_or_Department,Operating Status,0.859519,0.858586


In [20]:
op_status_ml = op_status_results[14][0]

In [21]:
X = combined_tfidf
y = le.fit_transform(budget_df['Operating_Status'])
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state = 42)

In [22]:
predict = op_status_ml.predict(X_val)

In [23]:
predict_proba = op_status_ml.predict_proba(X_val)

In [24]:
log_loss(y_val, predict_proba)

0.045826521097084155

In [25]:
# log_loss_scorer(op_status_ml, X_val, y_val)

In [50]:
test_tfidf = []
for col in budget_test[test_cols]:
#     tfidf = TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)
    col_tfidf = tfidf.transform(budget_test[col])
    test_tfidf.append(col_tfidf)

test_tfidf_combined = hstack(test_tfidf)

In [51]:
X_test = test_tfidf_combined

In [52]:
op_status_ml.predict(X_test)

ValueError: X has 1008 features per sample; expecting 4717

## Model predictions for all target columns

In [28]:
import itertools

In [53]:
model_results = []
for target, (tfidf, predictor) in itertools.product(targets, zip(tfidf_list, word_predictors)):
    X = tfidf
    y = le.fit_transform(budget_df[target])
    results = nlp_pipe(target, predictor, X, y)
    model_results.append(results)

In [54]:
score_results = []
for item in model_results:
    scores = item[1]
    score_results.append(scores)

In [55]:
scores_df = pd.DataFrame(score_results)

In [56]:
scores_df.sort_values('val_score', ascending=False)[:20]

,predictor,target,train_score,val_score
119,combined_tfidif,Pre_K,0.990082,0.989395
134,combined_tfidif,Operating_Status,0.985641,0.984748
104,combined_tfidif,Object_Type,0.986069,0.984523
89,combined_tfidif,Position_Type,0.983543,0.981688
59,combined_tfidif,Reporting,0.981538,0.980813
74,combined_tfidif,Student_Type,0.978434,0.975954
44,combined_tfidif,Sharing,0.973094,0.971795
29,combined_tfidif,Use,0.968937,0.966061
14,combined_tfidif,Function,0.960271,0.956106
116,Program_Description,Pre_K,0.954303,0.954294


In [ ]:
X = combined_tfidf
# for col in budget_df[targets]:
y = le.fit_transform(budget_df['Operating_Status'])
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state = 42)

In [ ]:
model_results[134][0].predict(X_train)

In [ ]:
model_results[134][0].predict_proba(X_train)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

In [57]:
pre_K_lr = model_results[119][0]
op_stat_lr = model_results[134][0]
ob_type_lr = model_results[104][0]
pos_type_lr = model_results[89][0]
reporting_lr = model_results[59][0]
student_lr = model_results[74][0]
sharing_lr = model_results[44][0]
use_lr = model_results[29][0]
func_lr = model_results[14][0]

In [58]:
from sklearn.externals import joblib

In [59]:
joblib.dump(pre_K_lr, 'pre_K.pkl')
joblib.dump(op_stat_lr, 'op_sta.pkl')
joblib.dump(ob_type_lr, 'ob_type.pkl')
joblib.dump(pos_type_lr, 'pos_type.pkl')
joblib.dump(reporting_lr, 'reporting.pkl')
joblib.dump(student_lr, 'student.pkl')
joblib.dump(sharing_lr, 'sharing.pkl')
joblib.dump(use_lr, 'use.pkl')
joblib.dump(func_lr, 'func.pkl')

['func.pkl']

## How to test with test set

In [30]:
test_tfidf = []a
for col in budget_test[test_cols]:
    tfidf = TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)
    col_tfidf = tfidf.fit_transform(budget_test[col])
    test_tfidf.append(col_tfidf)

test_tfidf_combined = hstack(test_tfidf)

In [31]:
X_test = test_tfidf_combined

In [32]:
test_tfidf_combined.get_shape()

(50064, 1484)

### stacked test tfidf number of columns different from train

## Figure out how to combine model of categorical columns and numerical

 running into memory issue for get_dummies for the 9 cat